## Modelo LDA para identificar temas en contratos Obra SECOP

### Cargar librerías

In [1]:
# Librerias

import csv
import pandas as pd
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.utils import deaccent


from nltk.corpus import stopwords
lista_stopwords = stopwords.words("spanish")

import spacy

## es_
nlp = spacy.load("es_core_news_md")

# Importadas por JP
import time
from pprint import pprint
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

### Funciones personalizadas

In [2]:
# Funciones

## Función para abrir e iterar el archivo de los contratos linea por linea

def iter_csv_file(filename, column_name):
    """
    Esta función toma un archivo CSV y el nombre de una columna y devuelve un iterador que
    produce los valores de esa columna para cada fila en el archivo.
    
    Argumentos:
        filename (str): El nombre del archivo CSV a leer.
        column_name (str): El nombre de la columna en la que queremos iterar.
        
    Yields:
        El valor de la columna especificada para cada fila en el archivo.
        
    Ejemplo de uso:
    
    >>> for value in iter_csv_file('datos.csv', 'edad'):
            print(value)
            
        27
        35
        42
        18
        ...
    """
    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            yield row[column_name]

## Función para iterar la fila de una columna de un dataframe linea por linea, se utiliza para generar el corpus

def iter_dataframe(df, column_name):
    """
    Esta función toma un DataFrame de pandas y el nombre de una columna y devuelve un iterador que
    produce los tokens lematizados para cada fila en la columna especificada.
    
    Argumentos:
        df (pandas.DataFrame): El DataFrame de pandas a leer.
        column_name (str): El nombre de la columna en la que queremos iterar.
        
    Yields:
        Una lista de los tokens lematizados para cada fila en la columna especificada.
    """
    for line in df[column_name]:
        # Tokeniza la línea utilizando simple_preprocess y se eliminan las palabras menores a 3 letras y los acentos
        tokens = simple_preprocess(line, deacc=True, min_len=3)
        # Se remueven las stopwords y las palabras que aparecen solo una vez antes de aplicar la lematización
        doc = [token for token in nlp(' '.join(tokens).lower()) if token.text not in lista_stopwords]
        yield [token.lemma_ for token in doc]

 
## Función para iterar una columna y devolver una lista de lemas se utiliza para generar el diccionario
           


def iter_column(df, col_name):
    """
    Esta función toma un DataFrame de pandas y el nombre de una columna y devuelve un iterador que
    produce una lista de lemas para cada línea en la columna especificada.
    
    Argumentos:
        df (pandas.DataFrame): El DataFrame de pandas a leer.
        col_name (str): El nombre de la columna en la que queremos iterar.
        
    Yields:
        Una lista de lemas para cada línea en la columna especificada.
        
    Ejemplo de uso:
    
    >>> for lemmas in iter_column(df, 'texto'):
            print(lemmas)
            
        ['comprar', 'manzana', 'pera', 'naranja']
        ['ir', 'cine', 'amigo']
        ['cocinar', 'comida', 'saludable', 'cena']
        ...
    """
    # Itera sobre cada línea en la columna especificada
    for line in df[col_name]:
        # Se eliminan los acentos de las palabras en la línea utilizando unidecode
        # Tokeniza la línea utilizando simple_preprocess
        tokens = simple_preprocess(line, deacc=True,min_len=3)
        # Se remueven las stopwords y las palabras que aparecen solo una vez antes de aplicar la lematización
        doc = [token for token in nlp(' '.join(tokens).lower()) if token.text not in lista_stopwords]
        # Itera sobre cada token en el objeto Doc y devuelve su forma lematizada utilizando el atributo lemma_
        lemmas = [token.lemma_ for token in doc]
        lemmas = [deaccent(lemma) for lemma in lemmas]
        # Genera una lista de lemas para cada línea en la columna de entrada utilizando la sentencia yield
        yield lemmas

  
## Función para crear el corpus a partir de todos los datos

class MyCorpus():
    """
    Esta clase es una implementación de la interfaz de corpus de Gensim, que define cómo se accede a los documentos en un corpus de texto.
    """
                
    # Constructor de la clase MyCorpus
    def __init__(self, dictionary):
        """
        Constructor de la clase MyCorpus.
        
        Argumentos:
        dictionary (gensim.corpora.Dictionary): Objeto de diccionario de Gensim que se utilizará para crear bolsas de palabras.
        """
        self.dictionary = dictionary

    # Método que devuelve un generador que produce bolsas de palabras para cada línea en el archivo CSV
    def __iter__(self):
        """
        Método que devuelve un generador que produce bolsas de palabras para cada línea en el archivo CSV.
        
        Yields:
        Una bolsa de palabras para cada línea en el archivo CSV.
        """
        # Itera sobre cada línea en el archivo CSV utilizando el método iter_csv_file
        for line in iter_csv_file('datos/df_secop_obra.csv', 'Detalle_Objeto_Contratar'):
            # Convierte la lista de lemas en una bolsa de palabras utilizando el método doc2bow de self.dictionary
            yield self.dictionary.doc2bow(line.split())


# Función para crear el corpus a partir de la muestra
class MyCorpus_sample():
    """
    Esta clase es una implementación de la interfaz de corpus de Gensim, que define cómo se accede a los documentos en un corpus de texto.
    """
                
    # Constructor de la clase MyCorpus
    def __init__(self, dictionary, df, column_name):
        """
        Constructor de la clase MyCorpus.
        
        Argumentos:
        dictionary (gensim.corpora.Dictionary): Objeto de diccionario de Gensim que se utilizará para crear bolsas de palabras.
        df (pandas.DataFrame): El DataFrame de pandas a leer.
        column_name (str): El nombre de la columna en el que queremos iterar.
        """
        self.dictionary = dictionary
        self.df = df
        self.column_name = column_name

    # Método que devuelve un generador que produce bolsas de palabras para cada línea en el dataframe
    def __iter__(self):
        """
        Método que devuelve un generador que produce bolsas de palabras para cada línea en el dataframe.
        
        Yields:
        Una bolsa de palabras para cada línea en el dataframe.
        """
        # Itera sobre cada línea en el dataframe utilizando el método iter_dataframe
        for line in iter_dataframe(self.df, self.column_name):
            # Convierte la lista de lemas en una bolsa de palabras utilizando el método doc2bow de self.dictionary
            yield self.dictionary.doc2bow(' '.join(line).split())


### Carga de datos - Objeto de contratos

In [19]:
# Carga de datos

## Carga de datos en en un dataframe
datos = pd.read_csv('../datos/df_secop_obra.csv',encoding='utf-8')

## Pasar la columna Detalle_Objeto_Contratar de object a string porque sino da un error
datos['Detalle_Objeto_Contratar']=datos['Detalle_Objeto_Contratar'].astype(str)


## Crear la muestra
datos_sample= datos.sample(n=10000, random_state=25) #42
#datos_sample= datos[datos['Objeto_Contratar'] == 'Servicios de Salud'].sample(n=1800, random_state=7) # Revisión JP
#datos_sample= datos[datos['Objeto_Contratar'] != 'Servicios de Edificación, Construcción de Instalaciones y Mantenimiento'].sample(n=1800, random_state=25) # Revisión JP

In [20]:
datos.groupby(['Objeto_Contratar']).count()['UID']

Objeto_Contratar
Alimentos, Bebidas y Tabaco                                                                                       53
Artículos Domésticos, Suministros y Productos Electrónicos de Consumo                                             50
Componentes y Equipos para Distribución y Sistemas de Acondicionamiento                                          604
Componentes y Suministros Electrónicos                                                                           478
Componentes y Suministros de Manufactura                                                                         493
Componentes y Suministros para Estructuras, Edificación, Construcción y Obras Civiles                          21290
Componentes, Accesorios y Suministros de Sistemas Eléctricos e Iluminación                                      2233
Difusión de Tecnologías de Información y Telecomunicaciones                                                      269
Equipo Médico, Accesorios y Suministros        

### Stopwords asociadas a departamentos y municipios

In [21]:
# Lectura de la información de archivo departamentos

df_dpto = pd.read_csv('../datos/Regiones_Departamentos.csv', sep=';')
df_dpto = list(iter_column(df_dpto, 'Dpto_SECOP'))
lista_dpto = []
for i in df_dpto:
    lista_dpto = lista_dpto + i
lista_dpto = list(set(lista_dpto))
    
# Lectura de la información de archivo municipios

df_municipio = pd.read_csv('../datos/Departamentos_y_municipios_de_Colombia.csv', sep=',')
df_municipio = list(iter_column(df_municipio, 'MUNICIPIO'))
lista_municipio = []
for i in df_municipio:
    lista_municipio = lista_municipio + i
lista_municipio = list(set(lista_municipio))

### Crear diccionaro - Retirar Stopword

In [22]:
## Crear el diccionario con la muestra

dictionary = corpora.Dictionary(iter_column(datos_sample, 'Detalle_Objeto_Contratar'))
#dictionary.filter_extremes(no_above=0.7) # Agregado por JP

once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]

departamento_list =  ['amazonas', 'antioquia', 'arauca', 'atlantico', 'bolivar', 'boyaca', 'caldas', 'caqueta', 'casanare', 'cauca', 'cesar', 'choco', 'cordoba', 'cundinamarca', 'guainia', 'guaviare', 'huila', 'la_guajira', 'magdalena', 'meta', 'narino', 'norte_de_santander', 'putumayo','quindio', 'risaralda', 'san_andres_y_providencia', 'santander', 'sucre', 'tolima', 'valle_del_cauca', 'vaupes', 'vichada']

stoplist=['municipio', 'municipal', 'departamento', 'san', 'jose'] ##Incluir otras palabras
stoplist= stoplist + ['santa', 'corregimiento', 'norte', 'esfuerzo', 'aunar', 'locativa', 'sur', 'oriente', 'occidente',
                      'segun', 'jurisdiccion', 'barrio', 'etapa', 'casco', 'contrato', 'contratar'] # Agregadas por JP

stoplist= stoplist + lista_dpto + lista_municipio


#Generar una sola lista de palabras a filtrar
stoplist = stoplist+departamento_list

#Extraer los ids de las palabras de la listas de stoplist que coinciden con las palabras del diccionario
stop_ids = [
    dictionary.token2id[stopword]
    for stopword in stoplist
    if stopword in dictionary.token2id
]

#Funcion de filtrado

dictionary.filter_tokens( once_ids+stop_ids)

print(dictionary)

Dictionary<4864 unique tokens: ['aceite', 'ampliacion', 'canal', 'cascado', 'cerramiento']...>


### Crear corpus

In [23]:
## Crear corpus con la muestra
corpus_sample= MyCorpus_sample(dictionary, datos_sample,'Detalle_Objeto_Contratar' )

### Modelo LDA

In [24]:
# Medir tiempo de inicio
start = time.time()

# Modelo Simple
Estimacion=LdaModel(corpus_sample, num_topics=7, id2word=dictionary, passes=10, eval_every = None)

top_topics = Estimacion.top_topics(corpus_sample)
num_topics=5

# Imprimir tiempo de procesamiento
end = time.time()
print("Tiempo de ejecución: " +str(end - start))


#pprint(top_topics)
pprint(Estimacion.print_topics())

Tiempo de ejecución: 639.2237668037415
[(0,
  '0.029*"servicio" + 0.018*"proyecto" + 0.016*"obra" + 0.015*"administrativo" '
  '+ 0.014*"realizar" + 0.013*"ejecucion" + 0.013*"tecnico" + '
  '0.012*"desarrollo" + 0.010*"contratista" + 0.009*"prestacion"'),
 (1,
  '0.050*"construccion" + 0.050*"red" + 0.043*"acueducto" + '
  '0.037*"alcantarillado" + 0.033*"sistema" + 0.030*"obra" + '
  '0.019*"ampliacion" + 0.016*"optimizacion" + 0.015*"tratamiento" + '
  '0.014*"sanitario"'),
 (2,
  '0.057*"mantenimiento" + 0.055*"adecuacion" + 0.052*"educativo" + '
  '0.043*"sede" + 0.034*"institucion" + 0.031*"construccion" + '
  '0.018*"mejoramiento" + 0.017*"centro" + 0.014*"aula" + 0.013*"principal"'),
 (3,
  '0.079*"construccion" + 0.077*"calle" + 0.056*"carrera" + '
  '0.032*"mejoramiento" + 0.027*"via" + 0.027*"urbano" + 0.025*"sector" + '
  '0.022*"vereda" + 0.022*"vivienda" + 0.021*"rural"'),
 (4,
  '0.065*"construccion" + 0.042*"obra" + 0.021*"casa" + 0.018*"resguardo" + '
  '0.016*"indigen

### Ver resultados modelo LDA

In [25]:
# Visualizamos los resultados

pyLDAvis.enable_notebook()
LDA_visualization = gensimvis.prepare(Estimacion, list(corpus_sample), dictionary)
pyLDAvis.save_html(LDA_visualization, 'lda_7_temas_1800_XXX.HTML')

LDA_visualization

c:\Users\Jelb7\OneDrive - Universidad de los Andes\MIAD\Proyecto_grado\TrabajoFinalDeGraduacionMIAD\.venv\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.060755 -0.242820       1        1  21.336378
0      0.245500  0.128346       2        1  18.663653
5     -0.222009 -0.022651       3        1  15.084980
1     -0.018525  0.125817       4        1  12.774967
6      0.061991 -0.174290       5        1  12.060473
3     -0.239795  0.083701       6        1  11.614249
4      0.112083  0.101898       7        1   8.465302, topic_info=              Term         Freq        Total Category  logprob  loglift
79             via  1893.000000  1893.000000  Default  30.0000  30.0000
18           calle   853.000000   853.000000  Default  29.0000  29.0000
44       educativo  1062.000000  1062.000000  Default  28.0000  28.0000
45   mantenimiento  2637.000000  2637.000000  Default  27.0000  27.0000
6     construccion  3187.000000  3187.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
185       mediante    50.374534   249.574574   Topic7  -5.0819   0.8689
144         urbano    53.498711   789.057069   Topic7  -5.0217  -0.2220
24           civil    41.424849   145.054807   Topic7  -5.2775   1.2160
114        ubicado    45.568443   491.718885   Topic7  -5.1822   0.0905
11         primero    42.172572   235.405490   Topic7  -5.2596   0.7497

[403 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
238       1  0.092039    acceso
238       3  0.341859    acceso
238       5  0.006574    acceso
238       6  0.216949    acceso
238       7  0.335285    acceso
...     ...       ...       ...
424       3  0.991292      vias
669       7  0.971504      vida
16        2  0.159983  vivienda
16        6  0.759136  vivienda
16        7  0.078423  vivienda

[582 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 6, 2, 7, 4, 5])

In [49]:
dimension=[
"Educación",
"Niñez y Juventud",
"Trabajo",
"Salud",
"Vivienda"
]


In [27]:
subdumensiones=["Analfabetismo",
"Bajo logro educativo",
"Barreras a servicios para cuidado de la primera infancia",
"Barreras de acceso a servicios de salud",
"Desempleo de larga duración",
"Hacinamiento crítico",
"Inadecuada eliminación de excretas",
"Inasistencia escolar",
"Material inadecuado de paredes exteriores",
"Material inadecuado de pisos",
"Rezago escolar",
"Sin acceso a fuente de agua mejorada",
"Sin aseguramiento en salud",
"Trabajo infantil",
"Trabajo informal"
]

In [31]:
import gensim
from nltk.corpus import brown
model = gensim.models.Word2Vec(brown.sents())

In [32]:
model.save('brown.embedding')
new_model = gensim.models.Word2Vec.load('brown.embedding')

In [42]:


from gensim.models import Word2Vec
from nltk.corpus import brown

# Preprocesamiento de datos
corpus = brown.sents()
model = Word2Vec(sentences=corpus_sample,  window=5, min_count=5, workers=4)

In [55]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [56]:
def find_synonyms(word, model):
    try:
        return [x[0] for x in model.most_similar(word)]
    except:
        return []

In [65]:
dimensions = ["educación", "Ninez","Juventud", "trabajo", "salud", "vivienda"]
synonyms = {}
for dim in dimensions:
    dim_synonyms = set()
    for word in dim.split():
        dim_synonyms.update(find_synonyms(word, model))
    synonyms[dim] = dim_synonyms

In [66]:
synonyms

{'educación': {'calidad',
  'de_ser',
  'difícil',
  'en_cuanto',
  'ha_hecho',
  'hemos',
  'la_pesca',
  'que_hace',
  'sería',
  'tenemos'},
 'Ninez': {'Atención',
  'Emeterio',
  'Interian',
  'María_Luisa',
  'María_de_los',
  'María_del_Carmen',
  'Rosa_María',
  'Tranquilino',
  'Wilfrido',
  'Ángeles'},
 'Juventud': {'Camaguey',
  'Camaguey_Cuba',
  'Holguin_province',
  'Isla_de',
  'Isla_de_la',
  'La_Habana',
  'Muerte',
  'Pinar_del_Rio',
  'Pinar_del_Río',
  'Sancti_Spiritus'},
 'trabajo': {'aquí',
  'ellos',
  'es_muy',
  'estamos',
  'estoy',
  'hablar',
  'hecho',
  'la_gente',
  'quiero',
  'tengo'},
 'salud': {'aquí',
  'es_muy',
  'estas',
  'falta',
  'hecho',
  'mí',
  'por_que',
  'qué',
  'tienes',
  'trabajo'},
 'vivienda': set()}

In [81]:
import spacy
import numpy as np
from gensim.corpora.dictionary import Dictionary

# Cargar el modelo pre-entrenado en español
nlp = spacy.load('es_core_news_md')

# Definir la función para encontrar sinónimos de una palabra en el diccionario
def find_synonyms(word, dictionary):
    # Obtener la representación vectorial de la palabra utilizando el modelo pre-entrenado
    word_vec = nlp(word).vector
    # Encontrar las palabras más similares al vector de la palabra en el diccionario
    similar_words = dictionary.get_similarities(word_vec, topn=10)
    # Obtener las palabras correspondientes a los índices encontrados
    similar_words = [dictionary[i] for i, _ in similar_words]
    # Retornar las palabras similares encontradas
    return similar_words

# Definir la lista de palabras para encontrar sinónimos
dimension=[
    "Educación",
    "Niñez",
    "Juventud",
    "Trabajo",
    "Salud",
    "Vivienda"
]



# Encontrar los sinónimos de cada palabra en la lista y mostrarlos
for word in dimension:
    synonyms = find_synonyms(word, dictionary)
    print(f"Sinónimos de {word}: {synonyms}")



AttributeError: 'Dictionary' object has no attribute 'get_similarities'

In [86]:
import spacy
import gensim.downloader as api

# Descargar el modelo pre-entrenado en español
nlp = spacy.load('es_core_news_md')


# Descargar el modelo pre-entrenado en español
model = api.load('fasttext-wiki-news-subwords-300')

# Definir la lista de palabras
dimension = [
    "Educación",
    "infancia",
    "Juventud",
    "Trabajo",
    "Salud",
    "Vivienda"
]

# Función para encontrar sinónimos de una palabra
def find_synonyms(word, model):
    # Obtener el vector de la palabra
    word_vec = model[word]
    # Encontrar las palabras más similares al vector de la palabra en el modelo
    similar_words = model.similar_by_vector(word_vec, topn=10)
    # Obtener las palabras correspondientes a los índices encontrados
    similar_words = [sim_word[0] for sim_word in similar_words]
    return similar_words

# Encontrar los sinónimos de cada palabra en la lista y mostrarlos
for word in dimension:
    synonyms = find_synonyms(word, model)
    print(f"Sinónimos de {word}: {synonyms}")



Sinónimos de Educación: ['Educación', 'Educacion', 'Sección', 'Investigación', 'educación', 'Institución', 'Organización', 'Cooperación', 'Formación', 'Publicación']
Sinónimos de infancia: ['infancia', 'infância', 'Infancia', 'adolescencia', 'existencia', 'independencia', 'instancia', 'ignorancia', 'distância', 'distancia']
Sinónimos de Juventud: ['Juventud', 'Juvenil', 'Juventude', 'Deportiva', 'Ciclón', 'Juventudes', 'juventud', 'Gimnástico', 'Gimnástica', 'Deportivos']
Sinónimos de Trabajo: ['Trabajo', 'Trabajos', 'Trabajador', 'Trabajadores', 'Obrera', 'Obrero', 'Gestión', 'Político', 'Política', 'Mensaje']
Sinónimos de Salud: ['Salud', 'Saludo', 'Saludos', 'Palud', 'salud', 'Papá', 'Pobre', 'Abuela', 'Promesa', 'Noche']
Sinónimos de Vivienda: ['Vivienda', 'Viviendo', 'Tienda', 'Azienda', 'Liendo', 'Tiendas', 'Viviene', 'Encomienda', 'vivienda', 'azienda']
